In [ ]:
#FINAL FILE 19032025

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Navigate to the page->imdb 2024 action
#driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=action")
# Navigate to the page->imdb 2024 adventure
#driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=adventure")
# Navigate to the page->imdb 2024 animation
#driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=animation")
# Navigate to the page->imdb 2024 crime
#driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=crime")
# Navigate to the page->imdb 2024 mystery
#driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=mystery")

In [4]:
# FINAL ACTION->CLEANED DATA WITHOUT NAN
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Set up the webdriver:
driver = webdriver.Chrome()

# Navigate to the page
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=action")

# Initialize lists to store movie details
movienamelist = []
ratings = []
votes = []
durations = []

# Scroll and click "50 more" button to load pages
click_counter = 0
while True:
    try:
        # Click the "50 more" button
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')

        # Perform the click using ActionChains
        ActionChains(driver).move_to_element(button).perform()

        button.click()

        # Increase the click counter to ensure click is happening or not
        click_counter += 1
        print(f"Clicked {click_counter} times")

        # Wait for the new content to load
        time.sleep(3)

    except:
        # If the button is not found or any other error occurs (no more pages to load)
        print("No more pages to load or button is no longer available.")
        break

# Now scrape the data from all loaded pages
scrolling = True
last_len = 0
while scrolling:
    old_page_source = driver.page_source

    # Scroll down the page
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    # wait to load page
    time.sleep(2)

    # Wait until new movies are visible on the page
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3[class='ipc-title__text']")))

    # Find all movie titles on the page
    moviename_elements = driver.find_elements(By.CSS_SELECTOR, "h3[class='ipc-title__text']")
    rating_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--rating']")
    vote_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--voteCount']")
    duration_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'dli-title-metadata-item') and contains(text(), 'h')]")

    # If no new titles were added, stop scrolling
    if len(movienamelist) == last_len:  # No new data was loaded
        scrolling = False
    else:
        last_len = len(movienamelist)  # Update last_len to the current length of the list

    # Collect data for each movie
    for i in range(len(moviename_elements)):
        movienamelist.append(moviename_elements[i].text)

        # Add "NAN" if the data is missing
        if i < len(rating_elements):
            ratings.append(rating_elements[i].text)
        else:
            ratings.append("NAN")  # Add NAN if no rating is found

        if i < len(vote_elements):
            votes.append(vote_elements[i].text)
        else:
            votes.append("NAN")  # Add NAN if no vote is found

        if i < len(duration_elements):
            durations.append(duration_elements[i].text)
        else:
            durations.append("NAN")  # Add NAN if no duration is found

# Create a DataFrame to store the movie details
genres = ['Action'] * len(movienamelist)

action = pd.DataFrame({
    'Movie Name': movienamelist,
    'Rating': ratings,
    'Votes': votes,
    'Duration': durations,
    'Genre': genres
})

# Display the total number of movies scraped
print(f"Total Movies Found: {len(action)}")

# Clean the data and handle missing/incorrect values
for index, row in action.iterrows():
    try:
        # Movie Name: Remove index part (like 1. Dune: Part Two)
        movie_name = row['Movie Name']
        movie_name = movie_name.lstrip('0123456789. ').strip()
        
        # Handle Rating: Replace NaN with 0 if rating is NaN
        rating = row['Rating']
        if pd.isna(rating):
            rating = 0  # Replace NaN with 0
            
        # Handle Votes: Replace NaN with 0 if votes is NaN or invalid
        votes = row['Votes']
        if pd.isna(votes) or votes == 'nan' or votes == '':
            votes = 0  # Set votes to 0 if it's NaN or invalid
        
        # Check and convert 'votes' to integer if it's in 'k' or 'M' format
        if isinstance(votes, str):
            votes = votes.replace('(', '').replace(')', '').strip().lower()
            
            if 'k' in votes:  # Check for 'k' indicating thousands
                number_part = votes.replace('k', '').strip()
                votes = int(float(number_part) * 1000)  # Convert to integer
            
            elif 'm' in votes:  # Check for 'm' indicating millions
                number_part = votes.replace('m', '').strip()
                votes = int(float(number_part) * 1000000)  # Convert to integer
            
            else:  # If no 'k' or 'm', directly convert to integer
                try:
                    votes = int(votes)
                except ValueError:
                    votes = 0  # If conversion fails, set votes to 0

        # Handle Duration: Convert to total minutes
        duration = row['Duration']
        if pd.notna(duration):  # If duration is not NaN
            duration = str(duration).strip().lower()  # Ensure it's a string

            hours = 0
            minutes = 0

            # Check if the duration contains 'h' (hours) and 'm' (minutes)
            if 'h' in duration:
                hours_part = duration.split('h')[0].strip()
                if hours_part.isdigit():  # Ensure it's a valid number
                    hours = int(hours_part)

            if 'm' in duration:
                minutes_part = duration.split('m')[0].strip().split()[-1]  # Handle cases where hours are present
                if minutes_part.isdigit():  # Ensure it's a valid number
                    minutes = int(minutes_part)

            total_minutes = hours * 60 + minutes
        else:
            total_minutes = 0  # If duration is NaN, set to 0

        # Replace cleaned values in DataFrame
        action.at[index, 'Movie Name'] = movie_name
        action.at[index, 'Rating'] = rating
        action.at[index, 'Votes'] = votes
        action.at[index, 'Duration'] = total_minutes
    except Exception as e:
        # Log the exception if any issue occurs while processing a row
        print(f"Error processing row {index}: {e}")
        continue  # Skip this row and move on to the next one

# Replace 'NAN' strings with actual NaN values in pandas
action.replace('NAN', pd.NA, inplace=True)

# Drop rows with NaN values in critical columns (Rating, Votes, Duration)
action.dropna(subset=['Rating', 'Votes', 'Duration'], how='any', inplace=True)

# Print the cleaned DataFrame
print(action)

# Save the cleaned movie details to a CSV file
if not action.empty:
    action.to_csv("actionfinal.csv", index=False)
    print("Cleaned CSV file has been saved successfully.")
else:
    print("No data to save after cleaning.")

# Close the driver
driver.quit()

Clicked 1 times
Clicked 2 times
Clicked 3 times
Clicked 4 times
Clicked 5 times
Clicked 6 times
Clicked 7 times
Clicked 8 times
Clicked 9 times
Clicked 10 times
Clicked 11 times
Clicked 12 times
Clicked 13 times
Clicked 14 times
Clicked 15 times
Clicked 16 times
Clicked 17 times
Clicked 18 times
Clicked 19 times
Clicked 20 times
Clicked 21 times
Clicked 22 times
Clicked 23 times
Clicked 24 times
Clicked 25 times
Clicked 26 times
No more pages to load or button is no longer available.
Total Movies Found: 1322


C:\Users\ASHWINI MURALIDHARAN\AppData\Local\Temp\ipykernel_78888\910017919.py:178: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  action.replace('NAN', pd.NA, inplace=True)


                Movie Name Rating   Votes  Duration   Genre
0        Kraven the Hunter    5.4   44000       127  Action
1             Gladiator II    6.6  216000       148  Action
2     Sonic the Hedgehog 3    6.9   52000       110  Action
3    Venom: The Last Dance    6.0  107000       110  Action
4           Dune: Part Two    8.5  612000       166  Action
..                     ...    ...     ...       ...     ...
881                Revenge    2.3      28        69  Action
882         Even the Score    8.0      11        66  Action
883                  Melek    5.9       7       113  Action
884        The Domino File    8.5       6        80  Action
885    Kallu Compound 1995    6.4       7        87  Action

[886 rows x 5 columns]
Cleaned CSV file has been saved successfully.


In [5]:
# FINAL ADVENTURE->CLEANED DATA WITHOUT NAN
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Set up the webdriver:
driver = webdriver.Chrome()

# Navigate to the page
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=adventure")

# Initialize lists to store movie details
movienamelist = []
ratings = []
votes = []
durations = []

# Scroll and click "50 more" button to load pages
click_counter = 0
while True:
    try:
        # Click the "50 more" button
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')

        # Perform the click using ActionChains
        ActionChains(driver).move_to_element(button).perform()

        button.click()

        # Increase the click counter to ensure click is happening or not
        click_counter += 1
        print(f"Clicked {click_counter} times")

        # Wait for the new content to load
        time.sleep(3)

    except:
        # If the button is not found or any other error occurs (no more pages to load)
        print("No more pages to load or button is no longer available.")
        break

# Now scrape the data from all loaded pages
scrolling = True
last_len = 0
while scrolling:
    old_page_source = driver.page_source

    # Scroll down the page
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    # wait to load page
    time.sleep(2)

    # Wait until new movies are visible on the page
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3[class='ipc-title__text']")))

    # Find all movie titles on the page
    moviename_elements = driver.find_elements(By.CSS_SELECTOR, "h3[class='ipc-title__text']")
    rating_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--rating']")
    vote_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--voteCount']")
    duration_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'dli-title-metadata-item') and contains(text(), 'h')]")

    # If no new titles were added, stop scrolling
    if len(movienamelist) == last_len:  # No new data was loaded
        scrolling = False
    else:
        last_len = len(movienamelist)  # Update last_len to the current length of the list

    # Collect data for each movie
    for i in range(len(moviename_elements)):
        movienamelist.append(moviename_elements[i].text)

        # Add "NAN" if the data is missing
        if i < len(rating_elements):
            ratings.append(rating_elements[i].text)
        else:
            ratings.append("NAN")  # Add NAN if no rating is found

        if i < len(vote_elements):
            votes.append(vote_elements[i].text)
        else:
            votes.append("NAN")  # Add NAN if no vote is found

        if i < len(duration_elements):
            durations.append(duration_elements[i].text)
        else:
            durations.append("NAN")  # Add NAN if no duration is found

# Create a DataFrame to store the movie details
genres = ['Adventure'] * len(movienamelist)

adventure = pd.DataFrame({
    'Movie Name': movienamelist,
    'Rating': ratings,
    'Votes': votes,
    'Duration': durations,
    'Genre': genres
})

# Display the total number of movies scraped
print(f"Total Movies Found: {len(adventure)}")

# Clean the data and handle missing/incorrect values
for index, row in adventure.iterrows():
    try:
        # Movie Name: Remove index part (like 1. Dune: Part Two)
        movie_name = row['Movie Name']
        movie_name = movie_name.lstrip('0123456789. ').strip()
        
        # Handle Rating: Replace NaN with 0 if rating is NaN
        rating = row['Rating']
        if pd.isna(rating):
            rating = 0  # Replace NaN with 0
            
        # Handle Votes: Replace NaN with 0 if votes is NaN or invalid
        votes = row['Votes']
        if pd.isna(votes) or votes == 'nan' or votes == '':
            votes = 0  # Set votes to 0 if it's NaN or invalid
        
        # Check and convert 'votes' to integer if it's in 'k' or 'M' format
        if isinstance(votes, str):
            votes = votes.replace('(', '').replace(')', '').strip().lower()
            
            if 'k' in votes:  # Check for 'k' indicating thousands
                number_part = votes.replace('k', '').strip()
                votes = int(float(number_part) * 1000)  # Convert to integer
            
            elif 'm' in votes:  # Check for 'm' indicating millions
                number_part = votes.replace('m', '').strip()
                votes = int(float(number_part) * 1000000)  # Convert to integer
            
            else:  # If no 'k' or 'm', directly convert to integer
                try:
                    votes = int(votes)
                except ValueError:
                    votes = 0  # If conversion fails, set votes to 0

        # Handle Duration: Convert to total minutes
        duration = row['Duration']
        if pd.notna(duration):  # If duration is not NaN
            duration = str(duration).strip().lower()  # Ensure it's a string

            hours = 0
            minutes = 0

            # Check if the duration contains 'h' (hours) and 'm' (minutes)
            if 'h' in duration:
                hours_part = duration.split('h')[0].strip()
                if hours_part.isdigit():  # Ensure it's a valid number
                    hours = int(hours_part)

            if 'm' in duration:
                minutes_part = duration.split('m')[0].strip().split()[-1]  # Handle cases where hours are present
                if minutes_part.isdigit():  # Ensure it's a valid number
                    minutes = int(minutes_part)

            total_minutes = hours * 60 + minutes
        else:
            total_minutes = 0  # If duration is NaN, set to 0

        # Replace cleaned values in DataFrame
        adventure.at[index, 'Movie Name'] = movie_name
        adventure.at[index, 'Rating'] = rating
        adventure.at[index, 'Votes'] = votes
        adventure.at[index, 'Duration'] = total_minutes
    except Exception as e:
        # Log the exception if any issue occurs while processing a row
        print(f"Error processing row {index}: {e}")
        continue  # Skip this row and move on to the next one

# Replace 'NAN' strings with actual NaN values in pandas
adventure.replace('NAN', pd.NA, inplace=True)

# Drop rows with NaN values in critical columns (Rating, Votes, Duration)
adventure.dropna(subset=['Rating', 'Votes', 'Duration'], how='any', inplace=True)

# Print the cleaned DataFrame
print(adventure)

# Save the cleaned movie details to a CSV file
if not adventure.empty:
    adventure.to_csv("adventurefinal.csv", index=False)
    print("Cleaned CSV file has been saved successfully.")
else:
    print("No data to save after cleaning.")

# Close the driver
driver.quit()

Clicked 1 times
Clicked 2 times
Clicked 3 times
Clicked 4 times
Clicked 5 times
Clicked 6 times
Clicked 7 times
Clicked 8 times
Clicked 9 times
Clicked 10 times
Clicked 11 times
No more pages to load or button is no longer available.
Total Movies Found: 598


C:\Users\ASHWINI MURALIDHARAN\AppData\Local\Temp\ipykernel_78888\1106664353.py:178: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adventure.replace('NAN', pd.NA, inplace=True)


                                Movie Name Rating   Votes  Duration      Genre
0                                  Moana 2    6.7   87000       100  Adventure
1                                     Flow    7.9   58000        85  Adventure
2                             Gladiator II    6.6  216000       148  Adventure
3                     Sonic the Hedgehog 3    6.9   52000       110  Adventure
4                    Venom: The Last Dance    6.0  107000       110  Adventure
..                                     ...    ...     ...       ...        ...
409                           Wheelies Inc    8.5       6       115  Adventure
410                                Anytime    2.6       7        90  Adventure
411                            Mukhadrakon    8.5       6        87  Adventure
412              The Road to OVC Champions    6.4       7        82  Adventure
413  Penelope & Everly's Holiday Adventure    9.0       8        86  Adventure

[414 rows x 5 columns]
Cleaned CSV file has been sa

In [6]:
# FINAL ANIMATION ->CLEANED DATA WITHOUT NAN
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Set up the webdriver:
driver = webdriver.Chrome()

# Navigate to the page
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=animation")

# Initialize lists to store movie details
movienamelist = []
ratings = []
votes = []
durations = []

# Scroll and click "50 more" button to load pages
click_counter = 0
while True:
    try:
        # Click the "50 more" button
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')

        # Perform the click using ActionChains
        ActionChains(driver).move_to_element(button).perform()

        button.click()

        # Increase the click counter to ensure click is happening or not
        click_counter += 1
        print(f"Clicked {click_counter} times")

        # Wait for the new content to load
        time.sleep(3)

    except:
        # If the button is not found or any other error occurs (no more pages to load)
        print("No more pages to load or button is no longer available.")
        break

# Now scrape the data from all loaded pages
scrolling = True
last_len = 0
while scrolling:
    old_page_source = driver.page_source

    # Scroll down the page
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    # wait to load page
    time.sleep(2)

    # Wait until new movies are visible on the page
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3[class='ipc-title__text']")))

    # Find all movie titles on the page
    moviename_elements = driver.find_elements(By.CSS_SELECTOR, "h3[class='ipc-title__text']")
    rating_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--rating']")
    vote_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--voteCount']")
    duration_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'dli-title-metadata-item') and contains(text(), 'h')]")

    # If no new titles were added, stop scrolling
    if len(movienamelist) == last_len:  # No new data was loaded
        scrolling = False
    else:
        last_len = len(movienamelist)  # Update last_len to the current length of the list

    # Collect data for each movie
    for i in range(len(moviename_elements)):
        movienamelist.append(moviename_elements[i].text)

        # Add "NAN" if the data is missing
        if i < len(rating_elements):
            ratings.append(rating_elements[i].text)
        else:
            ratings.append("NAN")  # Add NAN if no rating is found

        if i < len(vote_elements):
            votes.append(vote_elements[i].text)
        else:
            votes.append("NAN")  # Add NAN if no vote is found

        if i < len(duration_elements):
            durations.append(duration_elements[i].text)
        else:
            durations.append("NAN")  # Add NAN if no duration is found

# Create a DataFrame to store the movie details
genres = ['Animation'] * len(movienamelist)

animation = pd.DataFrame({
    'Movie Name': movienamelist,
    'Rating': ratings,
    'Votes': votes,
    'Duration': durations,
    'Genre': genres
})

# Display the total number of movies scraped
print(f"Total Movies Found: {len(animation)}")

# Clean the data and handle missing/incorrect values
for index, row in animation.iterrows():
    try:
        # Movie Name: Remove index part (like 1. Dune: Part Two)
        movie_name = row['Movie Name']
        movie_name = movie_name.lstrip('0123456789. ').strip()
        
        # Handle Rating: Replace NaN with 0 if rating is NaN
        rating = row['Rating']
        if pd.isna(rating):
            rating = 0  # Replace NaN with 0
            
        # Handle Votes: Replace NaN with 0 if votes is NaN or invalid
        votes = row['Votes']
        if pd.isna(votes) or votes == 'nan' or votes == '':
            votes = 0  # Set votes to 0 if it's NaN or invalid
        
        # Check and convert 'votes' to integer if it's in 'k' or 'M' format
        if isinstance(votes, str):
            votes = votes.replace('(', '').replace(')', '').strip().lower()
            
            if 'k' in votes:  # Check for 'k' indicating thousands
                number_part = votes.replace('k', '').strip()
                votes = int(float(number_part) * 1000)  # Convert to integer
            
            elif 'm' in votes:  # Check for 'm' indicating millions
                number_part = votes.replace('m', '').strip()
                votes = int(float(number_part) * 1000000)  # Convert to integer
            
            else:  # If no 'k' or 'm', directly convert to integer
                try:
                    votes = int(votes)
                except ValueError:
                    votes = 0  # If conversion fails, set votes to 0

        # Handle Duration: Convert to total minutes
        duration = row['Duration']
        if pd.notna(duration):  # If duration is not NaN
            duration = str(duration).strip().lower()  # Ensure it's a string

            hours = 0
            minutes = 0

            # Check if the duration contains 'h' (hours) and 'm' (minutes)
            if 'h' in duration:
                hours_part = duration.split('h')[0].strip()
                if hours_part.isdigit():  # Ensure it's a valid number
                    hours = int(hours_part)

            if 'm' in duration:
                minutes_part = duration.split('m')[0].strip().split()[-1]  # Handle cases where hours are present
                if minutes_part.isdigit():  # Ensure it's a valid number
                    minutes = int(minutes_part)

            total_minutes = hours * 60 + minutes
        else:
            total_minutes = 0  # If duration is NaN, set to 0

        # Replace cleaned values in DataFrame
        animation.at[index, 'Movie Name'] = movie_name
        animation.at[index, 'Rating'] = rating
        animation.at[index, 'Votes'] = votes
        animation.at[index, 'Duration'] = total_minutes
    except Exception as e:
        # Log the exception if any issue occurs while processing a row
        print(f"Error processing row {index}: {e}")
        continue  # Skip this row and move on to the next one

# Replace 'NAN' strings with actual NaN values in pandas
animation.replace('NAN', pd.NA, inplace=True)

# Drop rows with NaN values in critical columns (Rating, Votes, Duration)
animation.dropna(subset=['Rating', 'Votes', 'Duration'], how='any', inplace=True)

# Print the cleaned DataFrame
print(animation)

# Save the cleaned movie details to a CSV file
if not animation.empty:
    animation.to_csv("animationfinal.csv", index=False)
    print("Cleaned CSV file has been saved successfully.")
else:
    print("No data to save after cleaning.")

# Close the driver
driver.quit()

Clicked 1 times
Clicked 2 times
Clicked 3 times
Clicked 4 times
Clicked 5 times
Clicked 6 times
Clicked 7 times
Clicked 8 times
No more pages to load or button is no longer available.
Total Movies Found: 404
                                         Movie Name Rating   Votes  Duration  \
0                                           Moana 2    6.7   87000       100   
1                                              Flow    7.9   58000        85   
2                                    The Wild Robot    8.2  152000       102   
3                             Mufasa: The Lion King    6.6   47000       118   
4    The Lord of the Rings: The War of the Rohirrim    6.3   28000       134   
..                                              ...    ...     ...       ...   
236                   Iris the Movie: Full Energy!!    5.4      13       109   
237                  Alex and Mia's Awesome Movie!!    6.0      14        65   
238             Bizarros Peixes das Fossas Abissais    7.2       6      

C:\Users\ASHWINI MURALIDHARAN\AppData\Local\Temp\ipykernel_78888\2186292021.py:178: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  animation.replace('NAN', pd.NA, inplace=True)


In [7]:
# FINAL CRIME->CLEANED DATA WITHOUT NAN
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Set up the webdriver:
driver = webdriver.Chrome()

# Navigate to the page
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=crime")

# Initialize lists to store movie details
movienamelist = []
ratings = []
votes = []
durations = []

# Scroll and click "50 more" button to load pages
click_counter = 0
while True:
    try:
        # Click the "50 more" button
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')

        # Perform the click using ActionChains
        ActionChains(driver).move_to_element(button).perform()

        button.click()

        # Increase the click counter to ensure click is happening or not
        click_counter += 1
        print(f"Clicked {click_counter} times")

        # Wait for the new content to load
        time.sleep(3)

    except:
        # If the button is not found or any other error occurs (no more pages to load)
        print("No more pages to load or button is no longer available.")
        break

# Now scrape the data from all loaded pages
scrolling = True
last_len = 0
while scrolling:
    old_page_source = driver.page_source

    # Scroll down the page
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    # wait to load page
    time.sleep(2)

    # Wait until new movies are visible on the page
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3[class='ipc-title__text']")))

    # Find all movie titles on the page
    moviename_elements = driver.find_elements(By.CSS_SELECTOR, "h3[class='ipc-title__text']")
    rating_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--rating']")
    vote_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--voteCount']")
    duration_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'dli-title-metadata-item') and contains(text(), 'h')]")

    # If no new titles were added, stop scrolling
    if len(movienamelist) == last_len:  # No new data was loaded
        scrolling = False
    else:
        last_len = len(movienamelist)  # Update last_len to the current length of the list

    # Collect data for each movie
    for i in range(len(moviename_elements)):
        movienamelist.append(moviename_elements[i].text)

        # Add "NAN" if the data is missing
        if i < len(rating_elements):
            ratings.append(rating_elements[i].text)
        else:
            ratings.append("NAN")  # Add NAN if no rating is found

        if i < len(vote_elements):
            votes.append(vote_elements[i].text)
        else:
            votes.append("NAN")  # Add NAN if no vote is found

        if i < len(duration_elements):
            durations.append(duration_elements[i].text)
        else:
            durations.append("NAN")  # Add NAN if no duration is found

# Create a DataFrame to store the movie details
genres = ['Crime'] * len(movienamelist)

crime = pd.DataFrame({
    'Movie Name': movienamelist,
    'Rating': ratings,
    'Votes': votes,
    'Duration': durations,
    'Genre': genres
})

# Display the total number of movies scraped
print(f"Total Movies Found: {len(crime)}")

# Clean the data and handle missing/incorrect values
for index, row in crime.iterrows():
    try:
        # Movie Name: Remove index part (like 1. Dune: Part Two)
        movie_name = row['Movie Name']
        movie_name = movie_name.lstrip('0123456789. ').strip()
        
        # Handle Rating: Replace NaN with 0 if rating is NaN
        rating = row['Rating']
        if pd.isna(rating):
            rating = 0  # Replace NaN with 0
            
        # Handle Votes: Replace NaN with 0 if votes is NaN or invalid
        votes = row['Votes']
        if pd.isna(votes) or votes == 'nan' or votes == '':
            votes = 0  # Set votes to 0 if it's NaN or invalid
        
        # Check and convert 'votes' to integer if it's in 'k' or 'M' format
        if isinstance(votes, str):
            votes = votes.replace('(', '').replace(')', '').strip().lower()
            
            if 'k' in votes:  # Check for 'k' indicating thousands
                number_part = votes.replace('k', '').strip()
                votes = int(float(number_part) * 1000)  # Convert to integer
            
            elif 'm' in votes:  # Check for 'm' indicating millions
                number_part = votes.replace('m', '').strip()
                votes = int(float(number_part) * 1000000)  # Convert to integer
            
            else:  # If no 'k' or 'm', directly convert to integer
                try:
                    votes = int(votes)
                except ValueError:
                    votes = 0  # If conversion fails, set votes to 0

        # Handle Duration: Convert to total minutes
        duration = row['Duration']
        if pd.notna(duration):  # If duration is not NaN
            duration = str(duration).strip().lower()  # Ensure it's a string

            hours = 0
            minutes = 0

            # Check if the duration contains 'h' (hours) and 'm' (minutes)
            if 'h' in duration:
                hours_part = duration.split('h')[0].strip()
                if hours_part.isdigit():  # Ensure it's a valid number
                    hours = int(hours_part)

            if 'm' in duration:
                minutes_part = duration.split('m')[0].strip().split()[-1]  # Handle cases where hours are present
                if minutes_part.isdigit():  # Ensure it's a valid number
                    minutes = int(minutes_part)

            total_minutes = hours * 60 + minutes
        else:
            total_minutes = 0  # If duration is NaN, set to 0

        # Replace cleaned values in DataFrame
        crime.at[index, 'Movie Name'] = movie_name
        crime.at[index, 'Rating'] = rating
        crime.at[index, 'Votes'] = votes
        crime.at[index, 'Duration'] = total_minutes
    except Exception as e:
        # Log the exception if any issue occurs while processing a row
        print(f"Error processing row {index}: {e}")
        continue  # Skip this row and move on to the next one

# Replace 'NAN' strings with actual NaN values in pandas
crime.replace('NAN', pd.NA, inplace=True)

# Drop rows with NaN values in critical columns (Rating, Votes, Duration)
crime.dropna(subset=['Rating', 'Votes', 'Duration'], how='any', inplace=True)

# Print the cleaned DataFrame
print(crime)

# Save the cleaned movie details to a CSV file
if not crime.empty:
    crime.to_csv("crimefinal.csv", index=False)
    print("Cleaned CSV file has been saved successfully.")
else:
    print("No data to save after cleaning.")

# Close the driver
driver.quit()

Clicked 1 times
Clicked 2 times
Clicked 3 times
Clicked 4 times
Clicked 5 times
Clicked 6 times
Clicked 7 times
Clicked 8 times
Clicked 9 times
Clicked 10 times
Clicked 11 times
Clicked 12 times
Clicked 13 times
Clicked 14 times
Clicked 15 times
Clicked 16 times
Clicked 17 times
Clicked 18 times
Clicked 19 times
No more pages to load or button is no longer available.
Total Movies Found: 966


C:\Users\ASHWINI MURALIDHARAN\AppData\Local\Temp\ipykernel_78888\3250821250.py:178: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  crime.replace('NAN', pd.NA, inplace=True)


                            Movie Name Rating   Votes  Duration  Genre
0                         Emilia Pérez    5.4   84000       132  Crime
1                                 Trap    5.8  130000       105  Crime
2                             Longlegs    6.6  178000       101  Crime
3            Pushpa: The Rule - Part 2    6.1   54000       201  Crime
4                            The Order    6.8   38000       116  Crime
..                                 ...    ...     ...       ...    ...
655                  Mothers of Chibok    8.5       7       133  Crime
656                    Public Defender    8.0       9       140  Crime
657                     Even the Score    8.7       8       113  Crime
658            The Great British Heist    5.9       8        95  Crime
659  SMB-plush presents trick or death    9.2      18        85  Crime

[660 rows x 5 columns]
Cleaned CSV file has been saved successfully.


In [8]:
# FINAL MYSTERY->CLEANED DATA WITHOUT NAN
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Set up the webdriver:
driver = webdriver.Chrome()

# Navigate to the page
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=mystery")

# Initialize lists to store movie details
movienamelist = []
ratings = []
votes = []
durations = []

# Scroll and click "50 more" button to load pages
click_counter = 0
while True:
    try:
        # Click the "50 more" button
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')

        # Perform the click using ActionChains
        ActionChains(driver).move_to_element(button).perform()

        button.click()

        # Increase the click counter to ensure click is happening or not
        click_counter += 1
        print(f"Clicked {click_counter} times")

        # Wait for the new content to load
        time.sleep(3)

    except:
        # If the button is not found or any other error occurs (no more pages to load)
        print("No more pages to load or button is no longer available.")
        break

# Now scrape the data from all loaded pages
scrolling = True
last_len = 0
while scrolling:
    old_page_source = driver.page_source

    # Scroll down the page
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    # wait to load page
    time.sleep(2)

    # Wait until new movies are visible on the page
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3[class='ipc-title__text']")))

    # Find all movie titles on the page
    moviename_elements = driver.find_elements(By.CSS_SELECTOR, "h3[class='ipc-title__text']")
    rating_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--rating']")
    vote_elements = driver.find_elements(By.CSS_SELECTOR, "span[class='ipc-rating-star--voteCount']")
    duration_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'dli-title-metadata-item') and contains(text(), 'h')]")

    # If no new titles were added, stop scrolling
    if len(movienamelist) == last_len:  # No new data was loaded
        scrolling = False
    else:
        last_len = len(movienamelist)  # Update last_len to the current length of the list

    # Collect data for each movie
    for i in range(len(moviename_elements)):
        movienamelist.append(moviename_elements[i].text)

        # Add "NAN" if the data is missing
        if i < len(rating_elements):
            ratings.append(rating_elements[i].text)
        else:
            ratings.append("NAN")  # Add NAN if no rating is found

        if i < len(vote_elements):
            votes.append(vote_elements[i].text)
        else:
            votes.append("NAN")  # Add NAN if no vote is found

        if i < len(duration_elements):
            durations.append(duration_elements[i].text)
        else:
            durations.append("NAN")  # Add NAN if no duration is found

# Create a DataFrame to store the movie details
genres = ['Mystery'] * len(movienamelist)

mystery = pd.DataFrame({
    'Movie Name': movienamelist,
    'Rating': ratings,
    'Votes': votes,
    'Duration': durations,
    'Genre': genres
})

# Display the total number of movies scraped
print(f"Total Movies Found: {len(mystery)}")

# Clean the data and handle missing/incorrect values
for index, row in mystery.iterrows():
    try:
        # Movie Name: Remove index part (like 1. Dune: Part Two)
        movie_name = row['Movie Name']
        movie_name = movie_name.lstrip('0123456789. ').strip()
        
        # Handle Rating: Replace NaN with 0 if rating is NaN
        rating = row['Rating']
        if pd.isna(rating):
            rating = 0  # Replace NaN with 0
            
        # Handle Votes: Replace NaN with 0 if votes is NaN or invalid
        votes = row['Votes']
        if pd.isna(votes) or votes == 'nan' or votes == '':
            votes = 0  # Set votes to 0 if it's NaN or invalid
        
        # Check and convert 'votes' to integer if it's in 'k' or 'M' format
        if isinstance(votes, str):
            votes = votes.replace('(', '').replace(')', '').strip().lower()
            
            if 'k' in votes:  # Check for 'k' indicating thousands
                number_part = votes.replace('k', '').strip()
                votes = int(float(number_part) * 1000)  # Convert to integer
            
            elif 'm' in votes:  # Check for 'm' indicating millions
                number_part = votes.replace('m', '').strip()
                votes = int(float(number_part) * 1000000)  # Convert to integer
            
            else:  # If no 'k' or 'm', directly convert to integer
                try:
                    votes = int(votes)
                except ValueError:
                    votes = 0  # If conversion fails, set votes to 0

        # Handle Duration: Convert to total minutes
        duration = row['Duration']
        if pd.notna(duration):  # If duration is not NaN
            duration = str(duration).strip().lower()  # Ensure it's a string

            hours = 0
            minutes = 0

            # Check if the duration contains 'h' (hours) and 'm' (minutes)
            if 'h' in duration:
                hours_part = duration.split('h')[0].strip()
                if hours_part.isdigit():  # Ensure it's a valid number
                    hours = int(hours_part)

            if 'm' in duration:
                minutes_part = duration.split('m')[0].strip().split()[-1]  # Handle cases where hours are present
                if minutes_part.isdigit():  # Ensure it's a valid number
                    minutes = int(minutes_part)

            total_minutes = hours * 60 + minutes
        else:
            total_minutes = 0  # If duration is NaN, set to 0

        # Replace cleaned values in DataFrame
        mystery.at[index, 'Movie Name'] = movie_name
        mystery.at[index, 'Rating'] = rating
        mystery.at[index, 'Votes'] = votes
        mystery.at[index, 'Duration'] = total_minutes
    except Exception as e:
        # Log the exception if any issue occurs while processing a row
        print(f"Error processing row {index}: {e}")
        continue  # Skip this row and move on to the next one

# Replace 'NAN' strings with actual NaN values in pandas
mystery.replace('NAN', pd.NA, inplace=True)

# Drop rows with NaN values in critical columns (Rating, Votes, Duration)
mystery.dropna(subset=['Rating', 'Votes', 'Duration'], how='any', inplace=True)

# Print the cleaned DataFrame
print(mystery)

# Save the cleaned movie details to a CSV file
if not mystery.empty:
    mystery.to_csv("mysteryfinal.csv", index=False)
    print("Cleaned CSV file has been saved successfully.")
else:
    print("No data to save after cleaning.")

# Close the driver
driver.quit()

Clicked 1 times
Clicked 2 times
Clicked 3 times
Clicked 4 times
Clicked 5 times
Clicked 6 times
Clicked 7 times
Clicked 8 times
Clicked 9 times
Clicked 10 times
Clicked 11 times
Clicked 12 times
Clicked 13 times
No more pages to load or button is no longer available.
Total Movies Found: 662


C:\Users\ASHWINI MURALIDHARAN\AppData\Local\Temp\ipykernel_78888\2244833886.py:178: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mystery.replace('NAN', pd.NA, inplace=True)


             Movie Name Rating   Votes  Duration    Genre
0             Nosferatu    7.3  178000       132  Mystery
1                  Trap    5.8  130000       105  Mystery
2           Blink Twice    6.5  100000       102  Mystery
3               Smile 2    6.7   97000       127  Mystery
4    Paddington in Peru    6.7   17000       106  Mystery
..                  ...    ...     ...       ...      ...
431        Maid to Kill    8.5       8        77  Mystery
432  An Ordinary Killer    6.6      21       120  Mystery
433   Dai jia fa de ren    5.7       6        64  Mystery
434   Mesken (Messuage)    6.6       6        89  Mystery
435             Samhain    2.6       7        93  Mystery

[436 rows x 5 columns]
Cleaned CSV file has been saved successfully.


In [9]:
import pandas as pd

# actionfinal=pd.DataFrame()
# adventurefinal=pd.DataFrame()
# animationfinal=pd.DataFrame()
# crimefinal=pd.DataFrame()
# mysteryfinal=pd.DataFrame()

In [10]:
actiondf = pd.read_csv('actionfinal.csv')
adventuredf = pd.read_csv('adventurefinal.csv')
animationdf = pd.read_csv('animationfinal.csv')
crimedf = pd.read_csv('crimefinal.csv')
mysterydf = pd.read_csv('mysteryfinal.csv')

In [11]:
combinedf = pd.DataFrame()
for i in [actiondf,adventuredf,animationdf,crimedf,mysterydf]:
    combinedf = pd.concat([combinedf, i],axis=0) # axis=0 means vertical(row-wise)

In [15]:
# Save the combined DataFrame to a CSV file
combinedf.to_csv('combined_data.csv', index=False)

In [13]:
# combinedf=pd.concat(map(pd.read_csv,["actiondf","adventuredf","animationdf","crimedf","mysterydf"]),ignore_index=True)
# print(combinedf.head())

In [14]:
combinedf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2637 entries, 0 to 435
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  2637 non-null   object 
 1   Rating      2637 non-null   float64
 2   Votes       2637 non-null   int64  
 3   Duration    2637 non-null   int64  
 4   Genre       2637 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 123.6+ KB


In [16]:
actiondf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  886 non-null    object 
 1   Rating      886 non-null    float64
 2   Votes       886 non-null    int64  
 3   Duration    886 non-null    int64  
 4   Genre       886 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 34.7+ KB


In [17]:
adventuredf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  414 non-null    object 
 1   Rating      414 non-null    float64
 2   Votes       414 non-null    int64  
 3   Duration    414 non-null    int64  
 4   Genre       414 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 16.3+ KB


In [19]:
animationdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  241 non-null    object 
 1   Rating      241 non-null    float64
 2   Votes       241 non-null    int64  
 3   Duration    241 non-null    int64  
 4   Genre       241 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 9.5+ KB


In [20]:
crimedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  660 non-null    object 
 1   Rating      660 non-null    float64
 2   Votes       660 non-null    int64  
 3   Duration    660 non-null    int64  
 4   Genre       660 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 25.9+ KB


In [21]:
mysterydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  436 non-null    object 
 1   Rating      436 non-null    float64
 2   Votes       436 non-null    int64  
 3   Duration    436 non-null    int64  
 4   Genre       436 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 17.2+ KB
